In [11]:
import numpy as np
import pandas as pd
import pickle

In [12]:
#load data
df_ipca = pd.read_pickle("kelly_data_without_nanocap.p")
# impute and normalize
df_ipca.sort_values(by='eom', inplace=True, ignore_index=True)
df_ipca

,id,eom,isin,cusip,sedol,excntry,ret_exc_lead1m,ret_local_lead1m,ret_local,ret_exc,...,rmax5_rvol_21d,ni_be,ocf_at,ocf_at_chg1,mispricing_perf,mispricing_mgmt,qmj,qmj_prof,qmj_growth,qmj_safety
0,101096901,1962-01-31,None,None,None,USA,-0.056458,-0.054458,NaN,NaN,...,NaN,0.195541,0.155349,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100439001,1962-01-31,None,None,None,USA,0.134638,0.136638,NaN,NaN,...,NaN,NaN,0.162712,-0.002743,NaN,NaN,NaN,NaN,NaN,NaN
2,100367001,1962-01-31,None,None,None,USA,0.061176,0.063176,NaN,NaN,...,NaN,0.096779,0.113621,0.134063,NaN,NaN,NaN,NaN,NaN,NaN
3,100929901,1962-01-31,None,None,None,USA,0.084271,0.086271,NaN,NaN,...,NaN,0.018191,0.014136,0.345804,NaN,NaN,NaN,NaN,NaN,NaN
4,100557401,1962-01-31,None,None,None,USA,0.078000,0.080000,NaN,NaN,...,NaN,0.225732,0.028295,-0.176846,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2472336,103505601,2024-02-29,USH013011285,H01301128,None,USA,NaN,NaN,0.125682,0.120982,...,1.774894,0.021196,0.043651,-0.001643,0.714149,0.418649,1.074080,0.520855,0.242347,1.406096
2472337,103506201,2024-02-29,US89377M1099,89377M109,None,USA,NaN,NaN,-0.048618,-0.053318,...,1.051326,-0.282927,-0.036805,0.139385,0.427250,0.225884,-0.349948,-0.552488,1.664436,-1.236592
2472338,111110301,2024-02-29,US5679081084,567908108,None,USA,NaN,NaN,0.185714,0.181014,...,1.787254,0.112341,-0.091783,-0.148404,0.578259,0.143840,-0.825917,-0.042057,-0.584540,-0.440567
2472339,103503501,2024-02-29,US50050N1037,50050N103,None,USA,NaN,NaN,0.008359,0.003659,...,1.123519,0.615113,0.134312,0.081536,0.774603,0.425490,0.958723,1.413750,1.109947,-0.590045


In [13]:
df_ipca = df_ipca[df_ipca.ret_local<100].copy() # Remove extreme returns

In [14]:
df_new_factors = df_ipca.iloc[:,0:10].copy()

# Intermediate values

In [15]:
# Create intermediate factors
df_ipca["at"] = df_ipca.at_me * df_ipca.market_equity
df_ipca["sale"] = df_ipca.sale_me * df_ipca.market_equity
# should be cash + short term investment, but no short term investment is in delta
df_ipca["che"] = df_ipca.cash_at * df_ipca["at"]
# Should be investment and other advances, but in delta
df_ipca["ivao"] = 0
# Debt, not counting cash
df_ipca["debt"] = df_ipca.debt_me * df_ipca.market_equity
# No data
df_ipca["mib"] = 0
# No data
df_ipca["pstk"] = 0
# Common equity, in delta
df_ipca['ceq'] = 0

df_ipca["me_lag1"] = df_ipca.groupby('id')['market_equity'].shift(1)
df_ipca["at_lag1"] = df_ipca.groupby('id')['at'].shift(1)
df_ipca["noa_at_lag1"] = df_ipca.groupby('id')['noa_at'].shift(1)
df_ipca["dolvol_126d_lag1"] = df_ipca.groupby('id')['dolvol_126d'].shift(1)
df_ipca["ret_lag1"] = df_ipca.groupby('id')['ret_local'].shift(1) # Return from t-2 to t-1

# New Factors

In [16]:
df_new_factors['A2ME'] = df_ipca['at_me']
df_new_factors["AT"] = df_ipca["at"]
df_new_factors['ATO'] = df_ipca['sale'] / df_ipca["noa_at_lag1"]
df_new_factors['BEME'] = df_ipca["be_me"]
# Should be 1 year daily beta, but only downside beta is in the data
df_new_factors['Beta'] = df_ipca['betadown_252d']
# Should include short-term investment, but in delta
df_new_factors['C'] = df_ipca['cash_at']
df_new_factors['CTO'] = df_ipca['sale'] / df_ipca['at_lag1']
# No depreciation or amortization data
#df_new_factors['D2A'] = df_ipca['dp'] / df_ipca['at']
df_new_factors['DPI2A'] =  df_ipca["ppeinv_gr1a"] / df_ipca['at_lag1']
df_new_factors['E2P'] = df_ipca['ni_me']
# Should be overhead cost to sales, but only have R&D data
df_new_factors['FC2Y'] = df_ipca["rd_me"] * df_ipca.market_equity / df_ipca["sale"]
# FCF to book equity
df_new_factors['Free_CF'] = df_ipca["fcf_me"] / df_ipca["be_me"]
df_new_factors['Idio_vol'] = df_ipca['ivol_ff3_21d']
# Should be pct_change of AT, calculated from lagged AT
df_new_factors['Investment'] = (df_ipca['at'] - df_ipca["at_lag1"]) / df_ipca["at_lag1"]
# Liability / Assets
df_new_factors['Lev'] = 1 - 1 / df_ipca["at_be"]
df_new_factors[df_new_factors['Lev'] >= 0] # 160 weird observations where book equity > assets

df_new_factors['LME'] = df_ipca['me_lag1']
# Lagged 1 month volume ($) / market equity, but only has 6-month volume
df_new_factors['LTurnover'] = df_ipca['dolvol_126d_lag1'] / df_ipca['market_equity']
df_new_factors['NOA'] = df_ipca["noa_at"]
df_new_factors['OA'] = df_ipca["oaccruals_at"]
df_new_factors['OL'] = df_ipca["opex_at"]
df_new_factors['PCM'] = df_ipca["ebit_sale"]
# No depreciation data
#df_new_factors['PM'] = df_ipca['oiadp'] / df_ipca['sale']
df_new_factors['Prof'] = df_ipca['ope_be']
df_new_factors['Q'] = (df_ipca["market_equity"] + df_ipca["debt_me"] * df_ipca["market_equity"]) / df_ipca["at"]
df_new_factors['Rel_to_High'] = df_ipca['prc_highprc_252d']
df_new_factors['RNA'] = df_ipca['ebit_bev']
df_new_factors['ROA'] = df_ipca['niq_at']
df_new_factors['ROE'] = df_ipca['ni_be']
df_new_factors['r12_2'] = df_ipca["ret_12_1"] - df_ipca["ret_lag1"]
df_new_factors['r12_7'] = df_ipca['ret_12_7']
df_new_factors['r2_1'] = df_ipca["ret_lag1"]

r36_13 = 0 * df_ipca.ret_local # Initialize empty series
for i in range(13, 36):
    r36_13 = r36_13 + df_ipca.groupby('id')['ret_local'].shift(i)
df_new_factors['r36_13'] = r36_13

# Should be market cap of last December
df_new_factors['S2P'] = df_ipca['sale'] / df_ipca.market_equity
df_new_factors['SGA2S'] = (df_ipca['gp_at'] * df_ipca["at"] - df_ipca['ebit_sale'] * df_ipca['sale']) / df_ipca['sale']
df_new_factors['Spread'] = df_ipca['bidaskhl_21d']
# No predicted volume data
#df_new_factors['SUV'] = df_ipca['vol'] - df_ipca['predicted_volume']

In [17]:
inf_count = (df_new_factors==np.inf).sum().sum()
neg_inf_count = (df_new_factors==-np.inf).sum().sum()
df_new_factors_clean = df_new_factors.replace([np.inf, -np.inf], np.nan)
print(f"Replaced {inf_count} infinity and {neg_inf_count} negative infinity values with NaN")

Replaced 23471 infinity and 1052 negative infinity values with NaN


In [18]:
df_new_factors_clean.iloc[:,10:].max()

A2ME           1.741355e+03
AT             3.856741e+06
ATO            4.530117e+16
BEME           1.172480e+02
Beta           5.577180e+02
C              1.000000e+00
CTO            4.750000e+05
DPI2A          3.825000e+04
E2P            1.572028e+02
FC2Y           2.693060e+05
Free_CF        2.033400e+04
Idio_vol       5.485891e+02
Investment     2.639999e+06
Lev            9.999949e-01
LME            1.250928e+08
LTurnover      2.471802e+06
NOA            1.877000e+06
OA             1.689776e+02
OL             4.570000e+05
PCM            1.497121e+02
Prof           1.834200e+04
Q              1.516603e+06
Rel_to_High    1.000000e+00
RNA            2.927700e+04
ROA            1.208400e+04
ROE            1.847900e+04
r12_2          2.254716e+06
r12_7          1.526796e+06
r2_1           9.423810e+01
r36_13         9.485604e+01
S2P            5.518587e+02
SGA2S          3.717400e+05
Spread         1.496783e+00
dtype: float64

In [19]:
pickle.dump(df_new_factors_clean, open(f"no_nanocap_new_factors.p", "wb"))